### Import Packages
Just like the Python packages we are importing, we will import the Snowpark modules that we need.

In [ ]:
import json
from snowflake.snowpark import Session
import snowflake.snowpark.types as T

### Connect to Snowflake
Use these parameters and our Snowflake account credentials to connect to Snowflake and create a Snowpark session.

In [ ]:
# Get account credentials from a json file
with open("auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]
    f.close()

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "accountadmin",
    "warehouse": "demo_wh",
    "database": "demo",
    "schema": "public",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

### Upload CSVs
Upload product review CSVs to Snowflake internal stage.

In [ ]:
try:
    put_results = session.file.put(
        "product_reviews_*.csv",
        "@demo.public.files",
        overwrite=False,
        auto_compress=False)
    for r in put_results:
        str_output = ("File {src}: {status}").format(src = r.source, status=r.status)
        print(str_output)
except Exception as e:
    print(e)

### Create Snowpark DataFrame and write to Iceberg table

In [ ]:
# Define schema for CSV file
reviews_schema = T.StructType([T.StructField("ID", T.StringType()),
                               T.StructField("PRODUCT_NAME", T.StringType()),
                               T.StructField("PRODUCT_ID", T.StringType()),
                               T.StructField("REVIEWER_NAME", T.StringType()),
                               T.StructField("REVIEW_DATE", T.DateType()),
                               T.StructField("REVIEW", T.StringType()),
                               T.StructField("SENTIMENT", T.FloatType())])

In [ ]:
# Read January reviews CSV into DataFrame 
jan_df = session.read \
    .schema(reviews_schema) \
    .option("skip_header", 1) \
    .option("field_optionally_enclosed_by", '"') \
    .csv("@demo.public.files/product_reviews_jan_24.csv")

In [ ]:
# Preview January DataFrame
jan_df.show()

In [ ]:
# Write January product reviews to Snowflake-managed Iceberg Table
jan_df.write.mode("append").save_as_table("demo.public.product_reviews")

In [ ]:
# Read February reviews CSV into DataFrame 
feb_df = session.read \
    .schema(reviews_schema) \
    .option("skip_header", 1) \
    .option("field_optionally_enclosed_by", '"') \
    .csv("@demo.public.files/product_reviews_feb_24.csv")

In [ ]:
# Write February product reviews to Snowflake-managed Iceberg Table
feb_df.write.mode("append").save_as_table("demo.public.product_reviews")